# Capstone 2 - testing

compare every classifier using default parameters
select best classifiers and tune them
try feature engineering for low and high dollar
try feature importance


In [1]:
# Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# import seaborn as sns
import time

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
# import sklearn.model_selection as cv

# Classifiers
from sklearn.ensemble import (GradientBoostingClassifier, 
                              AdaBoostClassifier,
                              RandomForestClassifier)
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

# Measuring Recall & Precision
# from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import average_precision_score 
from sklearn.metrics import f1_score

import sys
sys.path.insert(0, '../src')
import helpers as h

In [ ]:
# Model Defaults:
SEED=123
FOLDS=5
CPU=-1

# Business Parameters:
FraudBudget=0.0005
ReviewCost=10
ChargebackFee=20

In [ ]:
# data = pd.read_csv("../data/creditcard.csv")
# features = ['V%d' % number for number in range(1, 29)] + ['Amount']

In [ ]:
# def feature_engineering(data):
#     df = data.copy()
#     df['LowAmount'] = (data['Amount'] <= 1)
#     df['LowAmount'] = df['LowAmount'].astype(int)
#     df['NonLowAmount'] = (data['Amount'] > 1)
#     df['NonLowAmount'] = df['NonLowAmount'].astype(int)
    
#     # since the population mean is zero, I converted the irrelevant features to zero

#     df['V15'] = df['LowAmount'] * data['V15']
#     df['V24'] = df['LowAmount'] * data['V24']
#     df['V26'] = df['LowAmount'] * data['V26']

#     df['V8'] = df['NonLowAmount'] * data['V8']
#     df['V19'] = df['NonLowAmount'] * data['V19']
#     df['V21'] = df['NonLowAmount'] * data['V21']

#     feature_lst = ['V%d' % number for number in range(1, 13)] + \
#                   ['V%d' % number for number in range(14, 22)] + \
#                   ['V24', 'V26']
#     return df, feature_lst

In [ ]:
def initialize_classifier_dict():
    """
    Returns a dictionary of classifier keys include colors for graphing
    Dictionary will later be populated with predictions and hyperparameters.
    """
    classifiers = { 
            "RF":  {"clf_desc": "RandomForest",
                    "model": RandomForestClassifier(n_jobs=CPU, 
                                                    random_state=SEED), 
                    "c": "g", 
                    "cmap": plt.cm.Greens,
                    "threshold": 0.5}

            ,"XGB": {"clf_desc": "XGBoost",
                    "model": XGBClassifier(n_jobs=CPU, random_state=SEED), 
                    "c": "blue", 
                    "cmap": plt.cm.Blues, 
                    "threshold": 0.5}
        
            ,"LR":  {"clf_desc": "LogisticRegression",
                    "model": LogisticRegression(n_jobs=CPU, 
                                              random_state = SEED), 
                    "c": "r", 
                    "cmap": plt.cm.Reds,
                    "threshold": 0.5}
            }
    
    # Make classifier pipelines:
    for clf in classifiers:
        steps = [# placeholder for smote: ('Preprocess', FeaturePreprocess), 
                 ('Classifier', classifiers[clf]['model'])]
        classifiers[clf]['pipeline'] = Pipeline(steps)    
    
    return classifiers





In [ ]:
# def test_classifier(clf):
#     '''
#     Fits the pipeline stored in classifiers dictionary on X_train, y_train data.
#     Makes predictions with pipiline on X_test, y_test data.  
#     Results are store in the classifiers dictionary.
#     '''     

#     model = classifiers[clf]['model'].fit(X_train, y_train)    
       
#     classifiers[clf]["y_pred"] = model.predict(X_test)
#     classifiers[clf]["test_Avg_Pr_score"] = average_precision_score(y_test, 
#                                                                classifiers[clf]["y_pred"])
#     classifiers[clf]["test_Recall_score"] = recall_score(y_test, 
#                                                     classifiers[clf]["y_pred"])
#     classifiers[clf]["test_Precision_score"] = precision_score(y_test, 
#                                                           classifiers[clf]["y_pred"])
#     classifiers[clf]["test_f1_score"] = f1_score(y_test, 
#                                             classifiers[clf]["y_pred"])

In [ ]:
# def cvtrain_classifier(clf, X, y):
#     '''
#     Runs cross validation on the pipeline stored in the classifiers dictionary under subkey['pipeline']
#     using X and y data. Results are store in the classifiers dictionary. 
#     Displays cross validation execution time if TimeIt=True.
#     '''     

#     cv_scores = cross_validate(classifiers[clf]['model'], X, y,
#                                 cv=StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED), 
#                                 return_train_score=False, 
#                                 scoring=["average_precision", "recall", "precision", "f1_macro"])

#     classifiers[clf]["cv_Avg_Pr_scores"] = cv_scores['test_average_precision']
#     classifiers[clf]["cv_Recall_scores"] = cv_scores['test_recall']
#     classifiers[clf]["cv_Precision_scores"] = cv_scores['test_precision']         
#     classifiers[clf]["cv_f1_scores"] = cv_scores['test_f1_macro']   
    
#     # use average to calculate a singel score:
#     classifiers[clf]["Avg_Pr_score"] = np.mean(classifiers[clf]["cv_Avg_Pr_scores"])
#     classifiers[clf]["Recall_score"] = np.mean(classifiers[clf]["cv_Recall_scores"])
#     classifiers[clf]["Precision_score"] = np.mean(classifiers[clf]["cv_Precision_scores"])
#     classifiers[clf]["f1_score"] = np.mean(classifiers[clf]["cv_f1_scores"])

In [ ]:
# def score_classifiers(clf_lst, TimeIt=True):
#     for clf in clf_lst:
#         start_time = time.time()

#         cvtrain_classifier(clf, X_train, y_train) 
#         test_classifier(clf)
        
#         if TimeIt:
#             t = time.time() - start_time
#             print(f"{t:.0f} seconds cross_validate execution time for {clf} classifier")
        

In [ ]:
# def plot_all_metrics(ax, clf_lst):
#     colors = []
#     clf_desc = []
#     for clf in clf_lst:
#         colors.append(classifiers[clf]["c"])
#         clf_desc.append(classifiers[clf]["clf_desc"])
#         results = pd.DataFrame.from_dict(classifiers, 
#                                          orient='index')[["clf_desc", 
#                                                         "Recall_score",
#                                                         "Precision_score",
#                                                         "f1_score",
#                                                         "Avg_Pr_score",]]    
#     results.set_index('clf_desc', inplace=True)
#     results.T.plot(kind='bar', color=colors, alpha=0.5, rot=0, ax=ax)
#     ax.set_ylim(ymin=0, ymax=1.0);
#     ax.grid('on', axis='y')
#     ax.legend(loc='lower right')
    

In [ ]:
fig, ax = plt.subplots()
ax = plot_all_metrics(ax, clf_lst)

In [ ]:
[classifiers[clf]["cv_Recall_scores"][0] for clf in classifiers]

In [ ]:
gs = gs.GridSpec(1,2)    
def plot_fraud_amount_eda(data=data, gs=gs):

    fig = plt.figure(figsize=(20,6))
    ax1 = fig.add_subplot(gs[0:-1])
    ax2 = fig.add_subplot(gs[1:])

    
grid = plt.GridSpec(2, 4, wspace=0.4, hspace=0.3)
plt.subplot(grid[:1, :1])
plt.subplot(grid[0, 2])
plt.subplot(grid[0, 3])
plt.subplot(grid[1, 2])
plt.subplot(grid[1, 3])

In [ ]:
function ROC_curve(probabilities, labels):
    Sort instances by their prediction strength (the probabilities)
    For every instance in increasing order of probability:
        Set the threshold to be the probability
        Set everything above the threshold to the positive class
        Calculate the True Positive Rate (aka sensitivity or recall)
        Calculate the False Positive Rate (1 - specificity)
    Return three lists: TPRs, FPRs, thresholds





model = LogisticRegression()
model.fit(X_train, y_train)
probabilities = model.predict_proba(X_test)[:, 1]

tpr, fpr, thresholds = roc_curve(probabilities, y_test)

plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate (1 - Specificity)")
plt.ylabel("True Positive Rate (Sensitivity, Recall)")
plt.title("ROC plot of Loan Interest Rate")
plt.show()

ISOLATION FOREST
If instances return a score very close to 1, then they are definitely anomalies.
If instances have a score much smaller than 0.5, then they are quite safe to be regarded as normal instances.
If all the instances return a score ≈ 0.5, then the entire sample does not really have any distinct anomaly.

You can set the contamination parameter to whatever percent your heart desires as long as it's a float in (0., 0.5).
"The amount of contamination of the data set, i.e. the proportion of outliers in the data set. Used when fitting to define the threshold on the decision function."
The default is 0.1 (or 10%), so you could set contamination=0.04 (4%).
from sklearn.ensemble import IsolationForest
clf = IsolationForest(contamination=0.04)

<!-- # Isolation Forest is unsupervised - just check it against the test set
#         The distance of the path is averaged and normalised to calculate 
#         the anomaly score. Anomaly score of 1 is considered as an outlier, 
#         values close to 0 is considered normal. -->

